In [1]:
from dotenv import load_dotenv
import os

# Cargar variables de entorno desde el archivo .env
load_dotenv()

# Asignar la clave de API a la variable de entorno que espera el SDK de Google
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

**Table of contents**<a id='toc0_'></a>    
- [Ingestión e indexación de la información](#toc1_)    
- [Almacenamiento de la información en base de datos vectorial](#toc2_)    
- [Construcción del retriever](#toc3_)    
- [Construcción del reranker](#toc4_)    
- [Generación de respuestas con RAG](#toc5_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[Ingestión e indexación de la información](#toc0_)

In [2]:
from langchain_text_splitters import MarkdownHeaderTextSplitter
from pathlib import Path

def load_and_chunk_markdown_documents(data_folder = "data"):
    
        """
    Carga todos los documentos Markdown desde la carpeta especificada,
    aplicando segmentación basada en estructura de encabezados para 
    generar chunks semánticamente coherentes, listos para el proceso 
    de embedding.

    Args:
        data_folder (str): Carpeta que contiene los archivos .md

    Returns:
        list: Lista de objetos Document (chunks) enriquecidos con metadatos
    """
    
        print("\nCARGA Y CHUNKING DE DOCUMENTOS EN FORMATO MARKDOWN")
        print("=" * 70)

        # Definir niveles jerárquicos que guiarán la segmentación
        headers_to_split_on = [
                ("#", "Header 1"),
                ("##", "Header 2"), 
                ("###", "Header 3"),
                ("####", "Header 4"),
        ]

        markdown_splitter = MarkdownHeaderTextSplitter(
                headers_to_split_on = headers_to_split_on,
                strip_headers = False
        )

        # Verificar existencia de la carpeta y archivos Markdown
        data_path = Path(data_folder)
        if not data_path.exists():
                raise FileNotFoundError(f"\nLa carpeta '{data_folder}' no existe")

        markdown_files = list(data_path.glob("*.md"))
        if not markdown_files:
                raise FileNotFoundError(f"\nNo se encontraron archivos .md en '{data_folder}'")

        all_chunks = []

        print(f"\nProcesando {len(markdown_files)} archivos:\n")
        print("-" * 30)

        for file_path in sorted(markdown_files):
                try:
                        # Leer contenido crudo del archivo
                        with open(file_path, 'r', encoding='utf-8') as file:
                                content = file.read()
                        
                        # Aplicar segmentación estructural
                        chunks = markdown_splitter.split_text(content)
                        
                        # Enriquecer cada chunk con metadatos útiles
                        for i, chunk in enumerate(chunks):
                                chunk.metadata.update({
                                        "source_file": file_path.name,
                                        "source_path": str(file_path),
                                        "chunk_index": i,
                                        "total_chunks_in_doc": len(chunks),
                                        "chunking_strategy": "document_structured",
                                        "file_size": file_path.stat().st_size,
                                        "chunk_size": len(chunk.page_content)
                                })
                        
                        all_chunks.extend(chunks)
                        print(f"{file_path.name}: {len(chunks)} chunks generados")

                except Exception as e:
                        print(f"\nError procesando {file_path.name}: {str(e)}")
                        continue

        print("-" * 30)
        print(f"Total chunks generados: {len(all_chunks)}")

        # Mostrar estadísticas generales
        if all_chunks:
                chunk_sizes = [len(chunk.page_content) for chunk in all_chunks]
                print(f"Tamaño promedio de chunks: {sum(chunk_sizes) / len(chunk_sizes):.0f} caracteres")
                print(f"Rango de tamaños: {min(chunk_sizes)} - {max(chunk_sizes)} caracteres")
                print("-" * 30)

        return all_chunks


def preview_chunks(chunks, num_examples = 5):
    
        """
    Muestra una vista previa de los primeros chunks generados.

    Args:
        chunks (list): Lista de objetos Document generados
        num_examples (int): Número de ejemplos a mostrar
    """
    
        print(f"\nVISTA PREVIA DE CHUNKS ({num_examples} ejemplos):")
        print("=" * 70)

        for i, chunk in enumerate(chunks[:num_examples]):
                print(f"\nChunk {i+1}:")
                print(f"Archivo: {chunk.metadata.get('source_file', 'unknown')}")
                print(f"Headers: {[f'{k}: {v}' for k, v in chunk.metadata.items() if 'Header' in k]}")
                print(f"Tamaño: {len(chunk.page_content)} caracteres")
                
                # Mostrar primeras líneas del contenido
                content_preview = chunk.page_content[:200].replace('\n', ' ')
                print(f"Contenido: '{content_preview}...'")
                print("-" * 40)

In [3]:
def prepare_documents_for_rag(data_folder = "data"):
    
        """
    Función completa que prepara documentos para el pipeline RAG.
    
    Returns:
        list: Chunks listos para crear embeddings y guardar en base de datos vectorial
    """
    
        # Cargar y hacer chunking de todos los documentos
        chunks = load_and_chunk_markdown_documents(data_folder)

        # Mostrar algunos ejemplos para validación rápida
        preview_chunks(chunks)

        return chunks


final_chunks = prepare_documents_for_rag()


CARGA Y CHUNKING DE DOCUMENTOS EN FORMATO MARKDOWN

Procesando 5 archivos:

------------------------------
api_development_standards.md: 20 chunks generados
development_policies.md: 28 chunks generados
multi_theme_descriptions.md: 7 chunks generados
software_architecture_guide.md: 14 chunks generados
troubleshooting_guide.md: 8 chunks generados
------------------------------
Total chunks generados: 77
Tamaño promedio de chunks: 413 caracteres
Rango de tamaños: 94 - 1692 caracteres
------------------------------

VISTA PREVIA DE CHUNKS (5 ejemplos):

Chunk 1:
Archivo: api_development_standards.md
Headers: ['Header 1: Estándares de Desarrollo de APIs', 'Header 2: Diseño RESTful', 'Header 3: Principios REST']
Tamaño: 567 caracteres
Contenido: '# Estándares de Desarrollo de APIs   ## Diseño RESTful   ### Principios REST REST (Representational State Transfer) es un estilo arquitectónico para servicios web que define las siguientes restriccion...'
----------------------------------------

C

# <a id='toc2_'></a>[Almacenamiento de la información en base de datos vectorial](#toc0_)

In [4]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_chroma import Chroma
import shutil

def load_to_chromadb(chunks, collection_name = "rag_docs", persist_directory = "./chroma_db"):
     
        """
    Carga una lista de chunks en una base de datos vectorial ChromaDB utilizando LangChain.
    
    Esta función prepara la colección eliminando cualquier contenido previo,
    genera embeddings para cada fragmento textual y los guarda junto con metadatos e identificadores únicos.
    
    Args:
        chunks (list): Lista de objetos Document generados previamente por el proceso de chunking.
        collection_name (str): Nombre lógico para la colección en la base de datos vectorial.
        persist_directory (str): Ruta local donde se almacenará la colección de forma persistente.

    Returns:
        Chroma: Objeto de almacenamiento vectorial que permite realizar consultas semánticas.
    """
    
        # Eliminar la base anterior si existe para asegurar una carga limpia
        if os.path.exists(persist_directory):
                shutil.rmtree(persist_directory)

        # Instanciar el modelo de embeddings de Google
        embeddings = GoogleGenerativeAIEmbeddings(model = "models/embedding-001")

        # Crear una base de datos vectorial persistente con la colección especificada
        vector_store = Chroma(
                collection_name = collection_name,
                embedding_function = embeddings,
                persist_directory = persist_directory
        )

        # Extraer los textos de cada chunk
        texts = [chunk.page_content for chunk in chunks]

        # Extraer los metadatos asociados (por ejemplo: archivo fuente, tamaño, headers)
        metadatas = [chunk.metadata for chunk in chunks]

        # Generar identificadores únicos para cada chunk
        ids = [f"{chunk.metadata.get('source_file', 'doc')}_{i}" for i, chunk in enumerate(chunks)]

        # Agregar los embeddings y metadatos a la colección
        vector_store.add_texts(texts = texts, metadatas = metadatas, ids = ids)

        print(f"\nChromaDB: {len(chunks)} chunks cargados")
        return vector_store

In [5]:
vector_store = load_to_chromadb(final_chunks)

E0000 00:00:1761256271.574200   39615 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.



ChromaDB: 77 chunks cargados


# <a id='toc3_'></a>[Construcción del retriever](#toc0_)

In [6]:
def create_retriever(vector_store, k = 3, search_type = "similarity"):
    
        """
    Crea un objeto Retriever a partir del vector store para el sistema RAG.
    
    Args:
        vector_store: Instancia de Chroma o cualquier base vectorial compatible.
        k (int): Número de documentos a recuperar por consulta.
        search_type (str): Tipo de búsqueda a realizar (por similitud, MMR, etc.)
    
    Returns:
        retriever: Objeto LangChain retriever listo para usar.
    """
    
        # Generar el retriever a partir del vector store con los parámetros indicados
        retriever = vector_store.as_retriever(
                search_type = search_type,
                search_kwargs = {"k": k}  # Número de resultados por consulta
        )

        # Mostrar configuración para verificación rápida
        print(f"Retriever: {search_type}, k = {k}")

        return retriever

In [7]:
retriever = create_retriever(vector_store, k = 10)

Retriever: similarity, k = 10


# <a id='toc4_'></a>[Construcción del reranker](#toc0_)

In [8]:
from sentence_transformers import CrossEncoder

def create_local_reranker(model_name = "cross-encoder/ms-marco-MiniLM-L-6-v2"):
    
        """
    Crea un objeto CrossEncoder para reranking local de documentos recuperados.

    Args:
        model_name (str): Nombre o ruta del modelo cross-encoder a utilizar.

    Returns:
        CrossEncoder: Objeto listo para usarse en el reranking.
    """
    
        # Incializacion del modelo de reranking
        cross_encoder = CrossEncoder(model_name)
        print(f"Reranker: {model_name}")
        
        return cross_encoder

In [9]:
cross_encoder = create_local_reranker()

config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

Reranker: cross-encoder/ms-marco-MiniLM-L-6-v2


In [10]:
def rerank_documents(reranker, question, retrieved_docs, n = 5, debug = False):
    
        """
    Reordena los documentos recuperados usando un modelo cross-encoder local.

    Args:
                reranker: Modelo utilizado para llevar a cabo el reranking.
        question (str): Pregunta del usuario.
        retrieved_docs (list): Lista de documentos recuperados inicialmente.
                n (int): Número de documentos a retornar

    Returns:
        list: Documentos reordenados por puntaje de relevancia descendente.
    """
    
        # Creación de todos los pares pregunta documento
        pairs = [(question, doc.page_content) for doc in retrieved_docs]

        # Calculo de puntajes de relevancia
        scores = reranker.predict(pairs)
        scored_docs = list(zip(retrieved_docs, scores))
        
        # Documentos ordenados por puntaje de relevancia
        reranked = sorted(scored_docs, key = lambda x: x[1], reverse = True)

        if debug:
                print("\nPUNTAJES ASIGNADOS POR EL RERANKER A LOS DOCUMENTOS RECUPERADOS POR EL RETRIVER:")
                print("=" * 70)

                for i, (doc, score) in enumerate(reranked, 1):
                        file = doc.metadata.get('source_file', 'unknown')  # Nombre del archivo original
                        preview = doc.page_content[:60]  # Fragmento del contenido
                        print("\n" + "-" * 40)
                        print(f"{i}. {file}: '{preview}...'")
                        print(f"   Relevance score: {score:.4f}")

                print("\n" + "-" * 40)
        
        return [doc for doc, _ in reranked[:n]]

In [11]:
def show_reranker(retriever, reranker, query):
    
        """
        Muestra el funcionamiento del reranker haciendo uso de una consulta.

        Args:
                retriever: Objeto retriever configurado previamente.
                reranker: Modelo utilizado para llevar a cabo el reranking
        query (str): Consulta de ejemplo para recuperar documentos.
    
        """

        # Ejecuta la consulta del usuario y recupera los documentos relevantes
        docs = retriever.invoke(query)

        # Aplica el proceso de reranking sobre los documentos recuperados
        reranked_docs = rerank_documents(reranker, query, docs, debug = True)

        # Se muestran los resultados cuando solo se usa el retriever + reranker
        print("\nRESULTADOS USANDO RETRIEVER + RERANKER:")
        print("=" * 70)

        # Muestra la consulta y la cantidad de documentos recuperados
        print(f"\nConsulta: '{query}'")
        print(f"Documentos recuperados: {len(reranked_docs)}")

        # Imprime una vista previa de cada documento recuperado
        for i, doc in enumerate(reranked_docs, 1):
                file = doc.metadata.get('source_file', 'unknown')  # Nombre del archivo original
                preview = doc.page_content[:60]  # Fragmento del contenido
                print("\n" + "-" * 30)
                print(f"\n  {i}. {file}: '{preview}...'")

        print("\n" + "-" * 30)

In [12]:
show_reranker(retriever, cross_encoder, "¿Qué prácticas ayudan a mejorar la robustez del manejo de errores y autenticación en APIs?")


PUNTAJES ASIGNADOS POR EL RERANKER A LOS DOCUMENTOS RECUPERADOS POR EL RETRIVER:

----------------------------------------
1. api_development_standards.md: '### Mejores Prácticas para Errores
- Proporcionar códigos de...'
   Relevance score: 0.6548

----------------------------------------
2. api_development_standards.md: '## Manejo de Errores  
### Estructura de Respuesta de Error
...'
   Relevance score: -1.4551

----------------------------------------
3. multi_theme_descriptions.md: '# Manual Técnico de Desarrollo de Software  
## Desarrollo d...'
   Relevance score: -3.8284

----------------------------------------
4. development_policies.md: '## 3. POLÍTICA DE SEGURIDAD EN EL CÓDIGO  
### 3.1 Manejo de...'
   Relevance score: -4.7480

----------------------------------------
5. multi_theme_descriptions.md: '## Gestión de Bases de Datos  
Los sistemas de bases de dato...'
   Relevance score: -4.7950

----------------------------------------
6. api_development_standards.md: '# Est

In [13]:
show_reranker(retriever, cross_encoder, "¿Cómo puedo implementar autenticación usando JWT?")


PUNTAJES ASIGNADOS POR EL RERANKER A LOS DOCUMENTOS RECUPERADOS POR EL RETRIVER:

----------------------------------------
1. api_development_standards.md: '## Autenticación y Autorización  
### JWT (JSON Web Tokens)
...'
   Relevance score: 0.8123

----------------------------------------
2. api_development_standards.md: '### Implementación de Autenticación
1. **Bearer Token**: `Au...'
   Relevance score: 0.5008

----------------------------------------
3. software_architecture_guide.md: '### Implementación
- Autenticación y autorización
- Cifrado ...'
   Relevance score: -1.2970

----------------------------------------
4. multi_theme_descriptions.md: '# Manual Técnico de Desarrollo de Software  
## Desarrollo d...'
   Relevance score: -2.1560

----------------------------------------
5. development_policies.md: '#### 3.1.2 Prácticas Obligatorias
**DEBE IMPLEMENTARSE**:
- ...'
   Relevance score: -9.5854

----------------------------------------
6. multi_theme_descriptions.md: '## C

# <a id='toc5_'></a>[Generación de respuestas con RAG](#toc0_)

In [14]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate

# Crear prompt personalizado en español.
# El LLM debe responder únicamente usando la información contextual recuperada.
prompt = ChatPromptTemplate.from_template("""
Eres un asistente especializado en responder preguntas sobre documentación técnica de desarrollo de software. Utiliza únicamente la información del contexto proporcionado para 
responder la pregunta. Si no conoces la respuesta basándote en el contexto, indica claramente que no tienes esa información. Mantén las respuestas concisas y precisas. Ignora
la información del contexto que no sea relevante para la pregunta en caso de que exista.
                                          
Pregunta: {question}

Contexto: {context}

Respuesta:
""")

# Configurar modelo de lenguaje de Google (gemini-2.5-flash)
# con baja temperatura para respuestas más deterministas.
llm = ChatGoogleGenerativeAI(
    model = "gemini-2.5-flash",
    temperature = 0.1,
    max_tokens = 300
)

E0000 00:00:1761256287.460744   39615 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


In [15]:
def ask_rag(question, retriever, reranker):
    
        """
    Ejecuta un flujo completo de RAG con reranking local:
    recuperación + reranking + generación.

    Args:
        question (str): Pregunta del usuario.
        retriever: Objeto retriever configurado previamente.
        reranker: Objeto CrossEncoder para reranking local.
        k (int): Número de fragmentos a recuperar inicialmente.

    Returns:
        dict: Resultado con pregunta, contexto, respuesta y fuentes.
    """
      
        # Paso 1: Recuperar documentos candidatos del retriever
        retrieved_docs = retriever.invoke(question)

        # Paso 2: Aplicar reranking local usando el cross-encoder
        reranked_docs = rerank_documents(reranker, question, retrieved_docs)

        # Paso 3: Preparar el contexto concatenando el contenido de los documentos reranqueados
        docs_content = "\\n\\n".join(doc.page_content for doc in reranked_docs)

        # Paso 4: Construir el mensaje de entrada usando el prompt definido
        messages = prompt.invoke({
                "question": question,
                "context": docs_content
        })

        # Paso 5: Generar la respuesta con el modelo LLM
        response = llm.invoke(messages)

        # Construir y devolver el resultado
        return {
                "question": question,
                "context": reranked_docs,
                "answer": response.content,
                "sources": [doc.metadata.get('source_file', 'unknown') for doc in reranked_docs]
        }

In [16]:
def display_rag_result(result):
    
        """
    Muestra los resultados de una ejecución RAG con formato estructurado.

    Args:
        result (dict): Diccionario con pregunta, contexto, respuesta y fuentes.
    """

        print("\n" + "=" * 60)
        print(f"🤔 PREGUNTA: {result['question']}")
        print("=" * 60)

        print(f"\n📚 CONTEXTO RECUPERADO:")
        print("-" * 30)
        for i, doc in enumerate(result["context"], 1):
                source = doc.metadata.get('source_file', 'unknown')
                preview = doc.page_content[:100].replace('\n', ' ')
                print("\n" + "-" * 30)
                print(f"{i}. {source}: '{preview}...'")

        print("\n" + "-" * 30)
        
        print(f"\n💡 RESPUESTA:")
        print("-" * 30)
        print(result["answer"])

        print(f"\n📄 FUENTES: {', '.join(set(result['sources']))}")

In [17]:
def test_rag_system(vector_store, reranker, test_questions = None):
    
        """
    Ejecuta una batería de pruebas sobre el sistema RAG.

    Args:
        vector_store: Vector store previamente cargado.
                reranker: Objeto CrossEncoder para reranking local.
        test_questions (list): Preguntas de prueba. Si no se especifican, se usan preguntas por defecto.
    """
      
        if test_questions is None:
                test_questions = [
                        "¿Qué son las APIs REST y cuáles son sus principios fundamentales?",
                        "¿Cómo funciona la autenticación JWT?",
                        "¿Cuáles son los códigos de estado HTTP más comunes?",
                        "¿Qué estrategias de chunking funcionan mejor para documentación técnica?",
                        "¿Cuáles son las mejores prácticas para el manejo de errores en APIs?"
                ]

        print("\n🧪 PRUEBAS DEL SISTEMA RAG")
        print("=" * 50)

        for i, question in enumerate(test_questions, 1):
                print(f"\n🔍 PRUEBA {i}/{len(test_questions)}")
                result = ask_rag(question, vector_store, reranker)
                display_rag_result(result)
                print("\n" + "=" * 60)

In [18]:
question = "\n¿Qué códigos HTTP se usan para errores del cliente?"
result = ask_rag(question, retriever, cross_encoder)
display_rag_result(result)

# Ejecución del conjunto completo de pruebas
print("\n🚀 INICIANDO PRUEBAS COMPLETAS...")
test_rag_system(retriever, cross_encoder)


🤔 PREGUNTA: 
¿Qué códigos HTTP se usan para errores del cliente?

📚 CONTEXTO RECUPERADO:
------------------------------

------------------------------
1. api_development_standards.md: '### Códigos de Error del Cliente (4xx) - **400 Bad Request**: Solicitud malformada - **401 Unauthori...'

------------------------------
2. api_development_standards.md: '### Mejores Prácticas para Errores - Proporcionar códigos de error específicos - Incluir mensajes de...'

------------------------------
3. multi_theme_descriptions.md: '# Manual Técnico de Desarrollo de Software   ## Desarrollo de APIs REST   Las APIs REST representan ...'

------------------------------
4. api_development_standards.md: '### Códigos de Error del Servidor (5xx) - **500 Internal Server Error**: Error interno del servidor ...'

------------------------------
5. api_development_standards.md: '## Códigos de Estado HTTP   ### Códigos de Éxito (2xx) - **200 OK**: Solicitud exitosa - **201 Creat...'

------------------------